In [29]:
# Dependencies and set up
import pandas as pd
import numpy as np
import json

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

# City of Chicago -- Crimes - from 2001 to 2021

In [30]:
# Raw crime table
table = pd.read_csv('/Users/karlaflores/Desktop/Git/Project-03/Crimes_-_2001_to_Present.csv') 
table

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,10224738,HY411648,09/05/2015 01:30:00 PM,043XX S WOOD ST,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True,...,12.0,61.0,08B,1165074.0,1875917.0,2015,02/10/2018 03:50:01 PM,41.815117,-87.670000,"(41.815117282, -87.669999562)"
1,10224739,HY411615,09/04/2015 11:30:00 AM,008XX N CENTRAL AVE,0870,THEFT,POCKET-PICKING,CTA BUS,False,False,...,29.0,25.0,06,1138875.0,1904869.0,2015,02/10/2018 03:50:01 PM,41.895080,-87.765400,"(41.895080471, -87.765400451)"
2,11646166,JC213529,09/01/2018 12:01:00 AM,082XX S INGLESIDE AVE,0810,THEFT,OVER $500,RESIDENCE,False,True,...,8.0,44.0,06,NaN,NaN,2018,04/06/2019 04:04:43 PM,NaN,NaN,NaN
3,10224740,HY411595,09/05/2015 12:45:00 PM,035XX W BARRY AVE,2023,NARCOTICS,POSS: HEROIN(BRN/TAN),SIDEWALK,True,False,...,35.0,21.0,18,1152037.0,1920384.0,2015,02/10/2018 03:50:01 PM,41.937406,-87.716650,"(41.937405765, -87.716649687)"
4,10224741,HY411610,09/05/2015 01:00:00 PM,0000X N LARAMIE AVE,0560,ASSAULT,SIMPLE,APARTMENT,False,True,...,28.0,25.0,08A,1141706.0,1900086.0,2015,02/10/2018 03:50:01 PM,41.881903,-87.755121,"(41.881903443, -87.755121152)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7372382,12436418,JE317243,07/27/2021 08:41:00 PM,007XX S LOOMIS ST,1130,DECEPTIVE PRACTICE,FRAUD OR CONFIDENCE GAME,APARTMENT,False,False,...,28.0,28.0,11,1167165.0,1896772.0,2021,08/03/2021 04:47:34 PM,41.872301,-87.661731,"(41.872300825, -87.661731237)"
7372383,12435900,JE316680,07/27/2021 10:45:00 PM,006XX N LA SALLE DR,0820,THEFT,$500 AND UNDER,GAS STATION,False,False,...,42.0,8.0,06,1175023.0,1904549.0,2021,08/03/2021 04:47:34 PM,41.893469,-87.632648,"(41.893469034, -87.632648229)"
7372384,12438154,JE319557,07/12/2021 09:00:00 AM,044XX W ADAMS ST,1320,CRIMINAL DAMAGE,TO VEHICLE,STREET,False,False,...,28.0,26.0,14,1146898.0,1898645.0,2021,08/03/2021 04:45:05 PM,41.877852,-87.736093,"(41.877851597, -87.736092822)"
7372385,12435637,JE316502,07/27/2021 05:01:00 PM,033XX W VAN BUREN ST,2024,NARCOTICS,POSSESS - HEROIN (WHITE),ALLEY,True,False,...,28.0,27.0,18,1153961.0,1897823.0,2021,08/03/2021 04:47:34 PM,41.875458,-87.710181,"(41.875458145, -87.710180976)"


In [31]:
# Groupby Primary type
df = table.groupby(['Primary Type']).count()['Description']
df

Primary Type
ARSON                                  12452
ASSAULT                               470968
BATTERY                              1351614
BURGLARY                              411063
CONCEALED CARRY LICENSE VIOLATION        767
CRIM SEXUAL ASSAULT                    27866
CRIMINAL DAMAGE                       840295
CRIMINAL SEXUAL ASSAULT                 3495
CRIMINAL TRESPASS                     207053
DECEPTIVE PRACTICE                    315360
DOMESTIC VIOLENCE                          1
GAMBLING                               14602
HOMICIDE                               11275
HUMAN TRAFFICKING                         73
INTERFERENCE WITH PUBLIC OFFICER       17679
INTIMIDATION                            4351
KIDNAPPING                              7032
LIQUOR LAW VIOLATION                   14529
MOTOR VEHICLE THEFT                   339751
NARCOTICS                             738637
NON - CRIMINAL                            38
NON-CRIMINAL                             1

In [32]:
# Groupby Primary type, year and arrest
df = table.groupby(['Year', 'Primary Type', 'Arrest']).count()['ID']
df

Year  Primary Type       Arrest
2001  ARSON              False       829
                         True        181
      ASSAULT            False     24234
                         True       7150
      BATTERY            False     72986
                                   ...  
2021  STALKING           True         11
      THEFT              False     18989
                         True        837
      WEAPONS VIOLATION  False      1824
                         True       3384
Name: ID, Length: 1244, dtype: int64

In [33]:
# Groupby Primary type, year and arrest and count by case number
df = table.groupby(['Year', 'Primary Type', 'Arrest']).count()['Case Number']
df

Year  Primary Type       Arrest
2001  ARSON              False       829
                         True        181
      ASSAULT            False     24234
                         True       7150
      BATTERY            False     72986
                                   ...  
2021  STALKING           True         11
      THEFT              False     18989
                         True        837
      WEAPONS VIOLATION  False      1824
                         True       3384
Name: Case Number, Length: 1244, dtype: int64

In [34]:
# Groupby Primary type and ID
df = table.groupby(['Primary Type', 'Arrest']).count()['ID']
df

Primary Type       Arrest
ARSON              False       10881
                   True         1571
ASSAULT            False      366627
                   True       104341
BATTERY            False     1048941
                              ...   
STALKING           True          611
THEFT              False     1370800
                   True       180179
WEAPONS VIOLATION  False       21570
                   True        69791
Name: ID, Length: 71, dtype: int64

In [35]:
len(df)

71

In [36]:
# Defining table type
table['Date'].dtypes

dtype('O')

In [37]:
# Crime column type
print (table.dtypes)

ID                        int64
Case Number              object
Date                     object
Block                    object
IUCR                     object
Primary Type             object
Description              object
Location Description     object
Arrest                     bool
Domestic                   bool
Beat                      int64
District                float64
Ward                    float64
Community Area          float64
FBI Code                 object
X Coordinate            float64
Y Coordinate            float64
Year                      int64
Updated On               object
Latitude                float64
Longitude               float64
Location                 object
dtype: object


In [38]:
# Dropping table
table = table.drop(columns=['Case Number', 'Location', 'Updated On', 'Block', 'Community Area', 'Beat', 'District', 'Ward', 'FBI Code', 'X Coordinate', 'Y Coordinate' ])
table

,ID,Date,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,Year,Latitude,Longitude
0,10224738,09/05/2015 01:30:00 PM,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True,2015,41.815117,-87.670000
1,10224739,09/04/2015 11:30:00 AM,0870,THEFT,POCKET-PICKING,CTA BUS,False,False,2015,41.895080,-87.765400
2,11646166,09/01/2018 12:01:00 AM,0810,THEFT,OVER $500,RESIDENCE,False,True,2018,NaN,NaN
3,10224740,09/05/2015 12:45:00 PM,2023,NARCOTICS,POSS: HEROIN(BRN/TAN),SIDEWALK,True,False,2015,41.937406,-87.716650
4,10224741,09/05/2015 01:00:00 PM,0560,ASSAULT,SIMPLE,APARTMENT,False,True,2015,41.881903,-87.755121
...,...,...,...,...,...,...,...,...,...,...,...
7372382,12436418,07/27/2021 08:41:00 PM,1130,DECEPTIVE PRACTICE,FRAUD OR CONFIDENCE GAME,APARTMENT,False,False,2021,41.872301,-87.661731
7372383,12435900,07/27/2021 10:45:00 PM,0820,THEFT,$500 AND UNDER,GAS STATION,False,False,2021,41.893469,-87.632648
7372384,12438154,07/12/2021 09:00:00 AM,1320,CRIMINAL DAMAGE,TO VEHICLE,STREET,False,False,2021,41.877852,-87.736093
7372385,12435637,07/27/2021 05:01:00 PM,2024,NARCOTICS,POSSESS - HEROIN (WHITE),ALLEY,True,False,2021,41.875458,-87.710181


In [39]:
# Dropping NAN from latitude
table = table[table['Latitude'].notna()]
table

,ID,Date,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,Year,Latitude,Longitude
0,10224738,09/05/2015 01:30:00 PM,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True,2015,41.815117,-87.670000
1,10224739,09/04/2015 11:30:00 AM,0870,THEFT,POCKET-PICKING,CTA BUS,False,False,2015,41.895080,-87.765400
3,10224740,09/05/2015 12:45:00 PM,2023,NARCOTICS,POSS: HEROIN(BRN/TAN),SIDEWALK,True,False,2015,41.937406,-87.716650
4,10224741,09/05/2015 01:00:00 PM,0560,ASSAULT,SIMPLE,APARTMENT,False,True,2015,41.881903,-87.755121
5,10224742,09/05/2015 10:55:00 AM,0610,BURGLARY,FORCIBLE ENTRY,RESIDENCE,False,False,2015,41.744379,-87.658431
...,...,...,...,...,...,...,...,...,...,...,...
7372382,12436418,07/27/2021 08:41:00 PM,1130,DECEPTIVE PRACTICE,FRAUD OR CONFIDENCE GAME,APARTMENT,False,False,2021,41.872301,-87.661731
7372383,12435900,07/27/2021 10:45:00 PM,0820,THEFT,$500 AND UNDER,GAS STATION,False,False,2021,41.893469,-87.632648
7372384,12438154,07/12/2021 09:00:00 AM,1320,CRIMINAL DAMAGE,TO VEHICLE,STREET,False,False,2021,41.877852,-87.736093
7372385,12435637,07/27/2021 05:01:00 PM,2024,NARCOTICS,POSSESS - HEROIN (WHITE),ALLEY,True,False,2021,41.875458,-87.710181


In [40]:
table.Date[0]

'09/05/2015 01:30:00 PM'

In [20]:
# Groupby by ID
table_2 = table.groupby(['Year', 'Primary Type', 'Arrest']).count()['ID']
table_2

Year  Primary Type       Arrest
2001  ARSON              False       824
                         True        180
      ASSAULT            False     24142
                         True       7118
      BATTERY            False     72665
                                   ...  
2021  STALKING           True         11
      THEFT              False     18518
                         True        837
      WEAPONS VIOLATION  False      1824
                         True       3383
Name: ID, Length: 1239, dtype: int64

In [21]:
# Crime evolution from 2001 to 2021
table_3 = table.groupby(['Year']).count()['ID']
table_3

Year
2001    482876
2002    471528
2003    472038
2004    467202
2005    449920
2006    445547
2007    435693
2008    420033
2009    386048
2010    370003
2011    351311
2012    335451
2013    306365
2014    273731
2015    257906
2016    267244
2017    264939
2018    263450
2019    259138
2020    208459
2021    109964
Name: ID, dtype: int64

In [22]:
# Table types
print (table.dtypes)

ID                        int64
Date                     object
IUCR                     object
Primary Type             object
Description              object
Location Description     object
Arrest                     bool
Domestic                   bool
Year                      int64
Latitude                float64
Longitude               float64
dtype: object


In [23]:
# IUCR json
url = 'https://data.cityofchicago.org/resource/c7ck-438e.json'
df_IUCR = pd.read_json(url)
df_IUCR

,iucr,primary_description,secondary_description,index_code
0,110,HOMICIDE,FIRST DEGREE MURDER,I
1,130,HOMICIDE,SECOND DEGREE MURDER,I
2,141,HOMICIDE,INVOLUNTARY MANSLAUGHTER,N
3,142,HOMICIDE,RECKLESS HOMICIDE,N
4,261,CRIM SEXUAL ASSAULT,AGGRAVATED: HANDGUN,I
...,...,...,...,...
396,5121,OTHER OFFENSE,ARSONIST: ANNUAL REGISTRATION,N
397,5122,OTHER OFFENSE,ARSONIST: FAIL TO REGISTER NEW ADDRESS,N
398,5130,OTHER OFFENSE,VIOLENT OFFENDER: DUTY TO REGISTER,N
399,5131,OTHER OFFENSE,VIOLENT OFFENDER: ANNUAL REGISTRATION,N


In [24]:
# Table types
print (df_IUCR.dtypes)

iucr                     object
primary_description      object
secondary_description    object
index_code               object
dtype: object


# Postgres connection - Chicago crime table

In [25]:
rds_connection_string = "postgres:123@localhost:5432/Project_03"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [26]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine,reflect=True)
Base

sqlalchemy.ext.automap.Base

In [27]:
engine.table_names()

['Chicago_crime']

In [28]:

table.to_sql(name='Chicago_crime', con=engine, if_exists='append', index=False)

KeyboardInterrupt: 

In [ ]:
from sqlalchemy import inspect

In [ ]:
inspector = inspect(engine)
inspector.get_columns('Chicago_crime')